In [1]:
from unsloth import tokenizer_utils
def do_nothing(*args, **kwargs):
    pass
tokenizer_utils.fix_untrained_tokens = do_nothing

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
import torch
major_version, minor_version = torch.cuda.get_device_capability()
print(f"Major: {major_version}, Minor: {minor_version}")
from datasets import load_dataset
import datasets
from trl import SFTTrainer
import pandas as pd
import numpy as np
import os
import pandas as pd
import numpy as np
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments, Trainer
from typing import Tuple
import warnings
from typing import Any, Dict, List, Union
from transformers import DataCollatorForLanguageModeling
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

#Models that have been tested
used_models = [
    "unsloth/Qwen3-0.6B-unsloth-bnb-4bit",
    "unsloth/Qwen3-1.7B-unsloth-bnb-4bit"
    "unsloth/Qwen3-4B-unsloth-bnb-4bit",   
    "unsloth/Qwen3-8B-unsloth-bnb-4bit",
    "unsloth/Qwen3-14B-unsloth-bnb-4bit"    
]

NUM_CLASSES = 2

max_seq_length = 2048 
dtype = None #None for auto detection

model_name = "unsloth/Qwen3-8B-unsloth-bnb-4bit";load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,load_in_4bit = load_in_4bit,
    max_seq_length = max_seq_length,
    dtype = dtype,
)

Major: 8, Minor: 9


==((====))==  Unsloth 2025.10.10: Fast Qwen3 patching. Transformers: 4.56.2.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 2. Max memory: 23.508 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
number_token_ids = []
for i in range(0, NUM_CLASSES+1):
    number_token_ids.append(tokenizer.encode(str(i), add_special_tokens=False)[0])
par = torch.nn.Parameter(model.lm_head.weight[number_token_ids, :])

old_shape = model.lm_head.weight.shape
old_size = old_shape[0]
print(par.shape)
print(old_shape)

model.lm_head.weight = par

reverse_map = {value: idx for idx, value in enumerate(number_token_ids)} 
reverse_map

torch.Size([3, 4096])
torch.Size([151936, 4096])


{15: 0, 16: 1, 17: 2}

In [4]:
from peft import LoftQConfig

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = [
        "lm_head",
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = True,
)
print("trainable parameters:", sum(p.numel() for p in model.parameters() if p.requires_grad))

Unsloth: Offloading output_embeddings to disk to save VRAM


/home/joaopedro/joaopedro/llm/.venv/lib/python3.12/site-packages/peft/tuners/tuners_utils.py:1222: UserWarning: Model has `tie_word_embeddings=True` and a tied layer is part of the adapter, but `ensure_weight_tying` is not set to True. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. Check the discussion here: https://github.com/huggingface/peft/issues/2777
  warnings.warn(msg)
Unsloth 2025.10.10 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


Unsloth: Training lm_head in mixed precision to save VRAM
trainable parameters: 43659264


In [ ]:
data = pd.read_csv("/home/joaopedro/joaopedro/llm/Mestrado/Dados/dataset_limpo.csv") #columns are text,label

train_df, val_df = train_test_split(data, test_size=0.2, random_state=42)
print(len(train_df))

token_counts = [len(tokenizer.encode(x)) for x in train_df.text]
a = plt.hist(token_counts, bins=30)

In [12]:
prompt = """Aqui está uma sentença.
{}

Se a sentença contém ideação suicida, responda "Sim". Caso contrário, responda "Não":

A resposta correta é: "{}"""

positivelabel = "Sim" #Yes
negativelabel = "Não" #No

def formatting_prompts_func(dataset_):
    texts = []
    for i in range(len(dataset_['text'])):
        text_ = dataset_['text'].iloc[i]
        label_ = dataset_['label'].iloc[i] 

        text = prompt.format(text_, label_)

        texts.append(text)
    return texts

#apply formatting_prompts_func to train_df
train_df['text'] = formatting_prompts_func(train_df)
train_dataset = datasets.Dataset.from_pandas(train_df,preserve_index=False)

In [13]:
class DataCollatorForLastTokenLM(DataCollatorForLanguageModeling):
    def __init__(
        self,
        *args,
        mlm: bool = False,
        ignore_index: int = -100,
        **kwargs,
    ):
        super().__init__(*args, mlm=mlm, **kwargs)
        self.ignore_index = ignore_index

    def torch_call(self, examples: List[Union[List[int], Any, Dict[str, Any]]]) -> Dict[str, Any]:
        batch = super().torch_call(examples)

        for i in range(len(examples)):
            last_token_idx = (batch["labels"][i] != self.ignore_index).nonzero()[-1].item()
            batch["labels"][i, :last_token_idx] = self.ignore_index
            batch["labels"][i, last_token_idx] = reverse_map[ batch["labels"][i, last_token_idx].item() ]

        return batch
collator = DataCollatorForLastTokenLM(tokenizer=tokenizer)

In [14]:
@torch.no_grad()
def predict_labels_llm(
    model,
    tokenizer,
    texts,
    max_seq_length,
    batch_size=32,
):
    model.eval()

    all_preds = []
    all_probs = []

    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]

        inputs = tokenizer(
            batch_texts,
            padding=True,
            truncation=True,
            max_length=max_seq_length,
            return_tensors="pt"
        ).to(model.device)

        outputs = model(**inputs)

        #logits do ÚLTIMO token
        logits = outputs.logits[:, -1, :2]

        probs = torch.softmax(logits, dim=-1)
        preds = torch.argmax(probs, dim=-1)

        all_probs.append(probs.cpu())
        all_preds.append(preds.cpu())

    return (
        torch.cat(all_preds).numpy(),
        torch.cat(all_probs).numpy()
    )

In [ ]:
import os
import json
import torch
import numpy as np

from transformers import TrainingArguments
from trl import SFTTrainer

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, precision_recall_curve
)
from datasets import Dataset

# =========================
# Configurações
# =========================
NUM_FOLDS = 5
SEED = 3407
BASE_OUTPUT_DIR = "/home/joaopedro/joaopedro/llm/Mestrado/Resultados/Teste_8B"

os.makedirs(BASE_OUTPUT_DIR, exist_ok=True)

torch.manual_seed(SEED)
np.random.seed(SEED)

# =========================
# Dataset
# =========================
full_dataset = Dataset.from_pandas(train_df, preserve_index=False)

skf = StratifiedKFold(
    n_splits=NUM_FOLDS,
    shuffle=True,
    random_state=SEED
)

all_fold_metrics = []

# =========================
# Cross-validation
# =========================
for fold, (train_idx, val_idx) in enumerate(
    skf.split(full_dataset["text"], full_dataset["label"])
):
    fold_id = fold + 1
    print(f"\n🚀 Fold {fold_id}/{NUM_FOLDS}")

    fold_dir = os.path.join(BASE_OUTPUT_DIR, f"fold_{fold_id}")
    os.makedirs(fold_dir, exist_ok=True)

    train_dataset = full_dataset.select(train_idx)
    val_dataset   = full_dataset.select(val_idx)

    # =========================
    # Treinamento
    # =========================
    training_args = TrainingArguments(
        per_device_train_batch_size=32,
        learning_rate=1e-4,
        num_train_epochs=1,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        output_dir=os.path.join(fold_dir, "model"),
        report_to="none",
        seed=SEED,
    )

    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=train_dataset,
        max_seq_length=max_seq_length,
        dataset_num_proc=1,
        args=training_args,
        data_collator=collator,
        dataset_text_field="text",
        formatting_func=formatting_prompts_func,
    )

    trainer.train()

    # =========================
    # Inferência (último token)
    # =========================
    y_true = np.array(val_dataset["label"])

    y_pred_raw, y_prob = predict_labels_llm(
        model,
        tokenizer,
        val_dataset["text"],
        max_seq_length=max_seq_length,
        batch_size=32,
    )

    y_score = y_prob[:, 1]  #probabilidade da classe positiva

    # =========================
    # Threshold ótimo (PR → F1)
    # =========================
    precision, recall, pr_thresholds = precision_recall_curve(y_true, y_score)

    f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
    optimal_idx = np.argmax(f1_scores[:-1])

    #pr_thresholds tem tamanho len(precision) - 1
    optimal_threshold = pr_thresholds[optimal_idx]

    y_pred = (y_score >= optimal_threshold).astype(int)

    # =========================
    # Métricas
    # =========================
    metrics = {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred, zero_division=0),
        "recall": recall_score(y_true, y_pred, zero_division=0),
        "f1": f1_score(y_true, y_pred, zero_division=0),
        "auc": roc_auc_score(y_true, y_score),
        "optimal_threshold": float(optimal_threshold),
    }

    print(f"🎯 Optimal Threshold: {optimal_threshold:.4f}")
    print(f"📊 Métricas Fold {fold_id}: {metrics}")

    all_fold_metrics.append(metrics)

    # =========================
    # Salvar métricas
    # =========================
    with open(os.path.join(fold_dir, "metrics.json"), "w") as f:
        json.dump(metrics, f, indent=4)

    # =========================
    # Curvas ROC e PR
    # =========================
    fpr, tpr, roc_thresholds = roc_curve(y_true, y_score)

    np.savez(
        os.path.join(fold_dir, "roc_pr_data.npz"),
        y_true=y_true,
        y_score=y_score,
        fpr=fpr,
        tpr=tpr,
        roc_thresholds=roc_thresholds,
        precision=precision,
        recall=recall,
        pr_thresholds=pr_thresholds,
    )

# =========================
# Média final
# =========================
final_metrics = {}
std_metrics   = {}

for key in all_fold_metrics[0]:
    values = [m[key] for m in all_fold_metrics]
    final_metrics[key] = float(np.mean(values))
    std_metrics[key]   = float(np.std(values))

print("\n✅ MÉTRICAS FINAIS (Cross-Validation)")
for k in final_metrics:
    print(f"{k.upper():15s}: {final_metrics[k]:.4f} ± {std_metrics[k]:.4f}")

summary = {
    "mean": final_metrics,
    "std": std_metrics,
    "per_fold": {
        f"fold_{i+1}": all_fold_metrics[i]
        for i in range(NUM_FOLDS)
    },
}

with open(os.path.join(BASE_OUTPUT_DIR, "summary_metrics.json"), "w") as f:
    json.dump(summary, f, indent=4)

Resultados Finais

In [1]:
import os
import json
import pandas as pd


MODEL_DIRS = {
    "Qwen-600M": "/home/joaopedro/joaopedro/llm/Mestrado/Resultados/results_Qwen_600M",
    "Qwen-1.7B": "/home/joaopedro/joaopedro/llm/Mestrado/Resultados/results_Qwen_1.7B",
    "Qwen-4B":   "/home/joaopedro/joaopedro/llm/Mestrado/Resultados/results_Qwen_4B",
    "Qwen-8B":   "/home/joaopedro/joaopedro/llm/Mestrado/Resultados/results_Qwen_8B_Teste1",
    "Qwen-14B":  "/home/joaopedro/joaopedro/llm/Mestrado/Resultados/results_Qwen_14BB",
}


rows = []

for model_name, base_dir in MODEL_DIRS.items():
    summary_path = os.path.join(base_dir, "summary_metrics.json")

    with open(summary_path, "r") as f:
        summary = json.load(f)

    mean_metrics = summary["mean"]
    std_metrics  = summary["std"]

    row = {
        "Model": model_name,
        "Accuracy (mean)": mean_metrics.get("accuracy"),
        "Accuracy (std)": std_metrics.get("accuracy"),
        "Precision (mean)": mean_metrics.get("precision"),
        "Precision (std)": std_metrics.get("precision"),
        "Recall (mean)": mean_metrics.get("recall"),
        "Recall (std)": std_metrics.get("recall"),
        "F1 (mean)": mean_metrics.get("f1"),
        "F1 (std)": std_metrics.get("f1"),
        "AUC (mean)": mean_metrics.get("auc"),
        "AUC (std)": std_metrics.get("auc"),
    }

    rows.append(row)

df_metrics = pd.DataFrame(rows)
df_metrics = df_metrics.sort_values("AUC (mean)", ascending=False)

print(df_metrics)

       Model  Accuracy (mean)  Accuracy (std)  Precision (mean)  \
4   Qwen-14B         0.988753        0.012297          0.976595   
0  Qwen-600M         0.980149        0.016460          0.959180   
1  Qwen-1.7B         0.966558        0.019576          0.936621   
2    Qwen-4B         0.960618        0.028433          0.901036   
3    Qwen-8B         0.955010        0.045170          0.898908   

   Precision (std)  Recall (mean)  Recall (std)  F1 (mean)  F1 (std)  \
4         0.027090       0.985136      0.016399   0.980769  0.020809   
0         0.044799       0.974791      0.009261   0.966526  0.026899   
1         0.036955       0.949504      0.051723   0.942137  0.035221   
2         0.063990       0.974831      0.021543   0.935894  0.044699   
3         0.102463       0.967967      0.030775   0.929382  0.065898   

   AUC (mean)  AUC (std)  
4    0.996465   0.002593  
0    0.992707   0.005327  
1    0.988740   0.007355  
2    0.980416   0.014141  
3    0.977704   0.027730  
